# Precision

`ebcc` supports single precision calculation, and this can be used to allow for mixed precision convergence, where a cheaper single precision calculation converges a set of amplitudes to use as a guess for a double precision calculation.

To perform a calculation in single precision, the context manager provided by `ebcc` can be used to handle the underlying casting and array initialisation. When using single precision, one typically will wish to relax the default convergence thresholds, since they may be less than machine precision for 32-bit floating points.

As many of the other features in `ebcc`, the precision control is designed to be agnostic to the method and ansatz, and should work seamlessly with any other functionality.

In [5]:
import numpy as np
from pyscf import gto, scf

mol = gto.M(atom="N 0 0 0; N 0 0 1.1", basis="cc-pvtz", verbose=0)
mf = scf.RHF(mol).run()

In [6]:
import sys
from logging import StreamHandler
from ebcc.core.logging import Logger

log = Logger("main")
log.setLevel(0)
log.addHandler(StreamHandler(sys.stdout))

In [18]:
from ebcc import REBCC
from ebcc.precision import single_precision

with single_precision():
    ccsd_sp = REBCC(mf, e_tol=1e-5, t_tol=1e-4, log=log)
    ccsd_sp.kernel()

    print("Energy dtype:", ccsd_sp.e_tot.dtype)
    print("T1 dtype:", ccsd_sp.t1.dtype)


RCCSD
*****

Options:
 > e_tol:  1e-05
 > t_tol:  0.0001
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (7o, 53v)

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.4000224173    -109.3830289495
   1    -0.3817788363    -109.3647853685     1.824e-02     1.898e-02
   2    -0.3978574574    -109.3808639896     1.608e-02     5.841e-03
   3    -0.3960854113    -109.3790919436     1.772e-03     2.377e-03
   4    -0.3979303539    -109.3809368861     1.845e-03     1.819e-03
   5    -0.3979354501    -109.3809419823     5.096e-06     3.083e-04
   6    -0.3979302943    -109.3809368265     5.156e-06     3.023e-05

Converged.

E(corr) = -0.3979302943
E(tot)  = -109.3809356689

Time elapsed: 794 ms

Energy dtype: float32
T1 dtype: float32


Care should be taken with attributes outside of the context, since those that are properties under the hood may perform some arithmetic that results in promotion since the context has exited.

In [19]:
print("Energy dtype (outside of context):", ccsd_sp.e_tot.dtype)

Energy dtype (outside of context): float64


Outside of the context manager, the initialised arrays and Hamiltonian will be double precision, so passing the 32-bit amplitudes should not be a problem as they will be quickly casted to 64-bit in the first iteration.

In [21]:
ccsd_dp = REBCC(mf, log=log)
ccsd_dp.amplitudes = ccsd_sp.amplitudes
ccsd_dp.kernel()

print("Energy dtype:", ccsd_dp.e_tot.dtype)
print("T1 dtype:", ccsd_dp.t1.dtype)


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (7o, 53v)

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.3979302835    -109.3809368157
   1    -0.3979367203    -109.3809432525     6.437e-06     7.550e-06
   2    -0.3979365774    -109.3809431096     1.429e-07     3.090e-06
   3    -0.3979372471    -109.3809437793     6.697e-07     3.101e-06
   4    -0.3979370159    -109.3809435482     2.311e-07     2.303e-07
   5    -0.3979371028    -109.3809436350     8.683e-08     1.623e-07
   6    -0.3979371068    -109.3809436390     4.019e-09     2.117e-08
   7    -0.3979371172    -109.3809436494     1.037e-08     1.491e-08
   8    -0.3979371163    -109.3809436485     9.048e-10     4.400e-08
   9    -0.3979371147    -109.3809436469     1.575e-09     9.038e-09

Converged.

E(corr) = -0.3979371147
E(tot)  = -109.3809436469

Time